## 데이터 불러오기

In [8]:
from autogluon.tabular import TabularDataset, TabularPredictor
import dask.dataframe as dd
import vaex
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import scipy.stats as spst
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.preprocessing import OneHotEncoder

In [9]:
card_data = '../data/five_city_data/merged_gyeonggi_day.csv'
flowpop_data = '../data/five_city_data/flowpop.csv'
commercial_area_ndc_data = '../data/five_city_data/updated_commerical_area_ndc.csv'
card_sample = '../data/gm_card.csv'

In [10]:
# card_df = dd.read_csv(card_sample,encoding='utf-8')

In [11]:
card_df = dd.read_csv(card_data,encoding='utf-8')
flowpop_df = dd.read_csv(flowpop_data,encoding='utf-8')
comm_df = dd.read_csv(commercial_area_ndc_data,encoding='utf-8')

## 데이터 전처리

#### 카드 소비 데이터

In [11]:
card_df.head()

,ta_ymd,cty_rgn_no,admi_cty_no,card_tpbuz_cd,card_tpbuz_nm_1,card_tpbuz_nm_2,hour,sex,age,day,amt,cnt
0,20230101,41210,41210510,D05,소매/유통,선물/완구,3,M,5,7,57576,2
1,20230101,41210,41210510,D11,소매/유통,종합소매점,3,F,7,7,8523,2
2,20230101,41210,41210510,D11,소매/유통,종합소매점,7,F,7,7,8023,2
3,20230101,41210,41210510,F02,생활서비스,미용서비스,3,M,4,7,28532,2
4,20230101,41210,41210510,F02,생활서비스,미용서비스,4,F,7,7,74074,2


In [12]:
flowpop_df.head()

,ETL_M,ADMI_CD,TIME_CD,M_10_CNT,F_10_CNT,M_20_CNT,F_20_CNT,M_30_CNT,F_30_CNT,M_40_CNT,F_40_CNT,M_50_CNT,F_50_CNT,M_60_CNT,F_60_CNT,M_70_CNT,F_70_CNT
0,2023-01,41111560,0,2858.84,2353.63,6248.39,4939.41,6684.28,4267.07,6304.54,5347.71,7495.55,5583.58,5411.89,4223.19,1086.22,1165.65
1,2023-01,41111560,1,3491.05,3021.34,7622.91,6183.19,7793.08,5365.17,7591.39,6765.04,9010.54,6842.45,6755.23,5363.00,1369.42,1499.24
2,2023-01,41111560,2,3433.43,3094.93,7369.46,5974.40,7510.31,5217.52,7282.29,6413.82,8717.53,6483.81,6631.32,5241.16,1303.01,1483.83
3,2023-01,41111560,3,3194.88,2955.40,7014.45,5704.24,7172.49,5138.34,6906.87,6334.15,8650.77,6395.10,6540.17,5316.28,1333.23,1501.99
4,2023-01,41111560,4,3003.63,2818.85,6716.24,5370.39,6980.61,4920.87,6908.53,6217.82,8911.40,6465.25,6940.02,5642.58,1519.33,1577.52


In [13]:
comm_df.head()

,행정동코드,기준분기,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분,상권변동지표구분명
0,41111560,1,99.9,46.4,HL,상권축소
1,41111566,1,83.5,43.2,LL,다이나믹
2,41111571,1,97.9,38.1,HL,상권축소
3,41111572,1,92.9,58.5,HH,정체
4,41111573,1,93.1,70.1,HH,정체


### 범주형 데이터 변환

In [44]:
# enc_classes = {} 
# def encoding_label(x):   # x: 범주형 타입의 컬럼(Series)
#     le = LabelEncoder()
#     le.fit(x)
#     label = le.transform(x)
#     enc_classes[x.name] = le.classes_   # x.name: 컬럼명
#     return label

In [58]:
# card_df2 = card_df['card_tpbuz_cd'].apply(encoding_label)
# 오늘, 행정동에서, 어떤 업종에 방문한 연령대의 남성이 발생시킨 매출의 평균
grouped_df = card_df.groupby(["ta_ymd", "admi_cty_no", "card_tpbuz_cd", "sex", "age"]).agg({"amt": "mean", "cnt": "sum"}).reset_index()

grouped_df.head(5)

,ta_ymd,admi_cty_no,card_tpbuz_cd,sex,age,amt,cnt
0,20230101,41210510,D05,M,5,57576.0,2
1,20230101,41210510,D11,F,7,8273.0,4
2,20230101,41210510,F02,F,4,97650.0,2
3,20230101,41210510,F02,F,6,25789.0,2
4,20230101,41210510,F02,F,7,74074.0,2


In [59]:
grouped_df = grouped_df.drop(['ta_ymd'],axis=1)

In [60]:
train_df, test_df = train_test_split(grouped_df, test_size=0.2, random_state=42)

In [61]:
# train_size = int(len(card_df) * 0.8)
# train_df = card_df[:train_size]
# test_df = card_df[train_size:]

In [62]:
predictor = TabularPredictor(
    label='amt',  # 예측할 타겟 컬럼 설정
    eval_metric='mean_absolute_error'  # 평가 지표 설정
)

No path specified. Models will be saved in: "AutogluonModels\ag-20240711_033255"


In [ ]:
predictor.fit(train_data=train_df)

In [64]:
# 예측 수행
predictions = predictor.predict(test_df)

In [66]:
predictions.head()

1018321    3.377258e+05
2124564    2.413185e+04
2258131    1.342081e+06
2011751    2.614102e+05
1738195    7.445822e+04
Name: amt, dtype: float32

In [67]:
test_df.head()

,admi_cty_no,card_tpbuz_cd,sex,age,amt,cnt
1018321,41210590,S04,F,6,3.442950e+04,4
2124564,41210520,Q14,M,2,3.107500e+04,1
2258131,41210610,S04,M,7,3.930405e+06,66
2011751,41210550,F02,F,7,4.016005e+05,16
1738195,41210580,F03,M,4,6.081390e+06,2


In [68]:
predictor.leaderboard(test_df)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTreesMSE,-185856.722470,-185911.330726,mean_absolute_error,2.764904,0.123686,201.497280,2.764904,0.123686,201.497280,1,True,7
1,RandomForestMSE,-188120.379728,-187931.481231,mean_absolute_error,2.519405,0.124849,262.785172,2.519405,0.124849,262.785172,1,True,5
2,LightGBM,-190159.331894,-189084.193637,mean_absolute_error,3.645283,0.145639,19.794451,3.645283,0.145639,19.794451,1,True,4
3,LightGBMXT,-192731.062032,-192396.499009,mean_absolute_error,83.806180,3.146824,177.869431,83.806180,3.146824,177.869431,1,True,3
4,CatBoost,-216138.837812,-212478.484948,mean_absolute_error,0.667849,0.080087,1044.560808,0.667849,0.080087,1044.560808,1,True,6
5,KNeighborsUnif,-415725.153611,-446119.955703,mean_absolute_error,16.806362,1.236200,14.697853,16.806362,1.236200,14.697853,1,True,1
6,KNeighborsDist,-416299.692563,-446152.890681,mean_absolute_error,15.315404,0.645500,2.131970,15.315404,0.645500,2.131970,1,True,2


## 데이터 결합

### 월 별 데이터로 통합, 유동인구와 상권변화지표도 통합

In [14]:
# 해당일, 해당동의 해당 업종의 특정시간에 매출과 매출건수 총액
card_df2 = card_df.groupby(["ta_ymd", "admi_cty_no", "card_tpbuz_cd"]).agg({"amt": "sum", "cnt":"sum"}).reset_index()

In [15]:
card_df2.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt
0,20230101,41210510,D05,57576,2
1,20230101,41210510,D11,16546,4
2,20230101,41210510,F02,313388,16
3,20230101,41210510,Q01,69266,1
4,20230101,41210510,Q15,78222,6


In [16]:
card_df2['ta_ymd'] = card_df2['ta_ymd'].astype(str) 

In [17]:
card_df2['ta_ymd'] = card_df2['ta_ymd'].str.slice(0, 6)

In [18]:
card_df2.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt
0,202301,41210510,D05,57576,2
1,202301,41210510,D11,16546,4
2,202301,41210510,F02,313388,16
3,202301,41210510,Q01,69266,1
4,202301,41210510,Q15,78222,6


In [19]:
# 월별 행정구별, 업종별, amt 및 cnt 평균
grouped_df = card_df2.groupby(["ta_ymd", "admi_cty_no", "card_tpbuz_cd"]).agg({"amt": "sum", "cnt": "sum"}).reset_index()
grouped_df.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt
0,202301,41210510,D05,633562,28
1,202301,41210510,D11,171040,28
2,202301,41210510,F02,12060758,375
3,202301,41210510,Q01,1818710,9
4,202301,41210510,Q15,2167723,87


In [20]:
# join을 위한 데이터 통일
flowpop_df['ETL_M'] = flowpop_df['ETL_M'].str.replace('-', '')
# 총 유동인구 계산
cnt_columns = flowpop_df.loc[:, 'M_10_CNT':'F_70_CNT'].columns
flowpop_df['TOTAL_POPULATION'] = flowpop_df[cnt_columns].sum(axis=1)
# 해당 월에 해당 동에 해당 시간에 방문한 유동인구수 평균
flowpop_df_temp = flowpop_df.groupby(["ETL_M", "ADMI_CD","TIME_CD"]).agg({"TOTAL_POPULATION": "mean"}).reset_index()
# 해당 월에 해당 동에 방문한 유동인구 평균 총합
flow_g_df = flowpop_df_temp.groupby(["ETL_M", "ADMI_CD"]).agg({"TOTAL_POPULATION": "sum"}).reset_index()

In [21]:
flow_g_df.head()

,ETL_M,ADMI_CD,TOTAL_POPULATION
0,202301,41111560,2968697.82
1,202301,41111566,5438074.73
2,202301,41111571,2764829.73
3,202301,41111572,3664691.19
4,202301,41111573,5278122.27


In [22]:
# 데이터 결합
merged_df = dd.merge(grouped_df, flow_g_df, how='inner', left_on=['ta_ymd', 'admi_cty_no'], right_on=['ETL_M', 'ADMI_CD'])


In [23]:
merged_df.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt,ETL_M,ADMI_CD,TOTAL_POPULATION
0,202301,41210510,D05,633562,28,202301,41210510,13850.98
1,202301,41210510,D11,171040,28,202301,41210510,13850.98
2,202301,41210510,F02,12060758,375,202301,41210510,13850.98
3,202301,41210510,Q01,1818710,9,202301,41210510,13850.98
4,202301,41210510,Q15,2167723,87,202301,41210510,13850.98


In [24]:
# 기준분기 생성 함수
def get_quarter(month):
    if month in [1, 2, 3]:
        return 1
    elif month in [4, 5, 6]:
        return 2
    elif month in [7, 8, 9]:
        return 3
    elif month in [10, 11, 12]:
        return 4

# merged_df에서 기준분기 생성
merged_df['year'] = merged_df['ta_ymd'].astype(str).str[:4].astype(int)
merged_df['month'] = merged_df['ta_ymd'].astype(str).str[4:6].astype(int)
merged_df['기준분기'] = merged_df['month'].apply(get_quarter)

# comm_df에서 기준분기와 매칭될 수 있도록 연도를 추가
comm_df['year'] = 2023

C:\Users\DA\anaconda3\envs\hanati\lib\site-packages\dask\dataframe\core.py:4336: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('month', 'int64'))

  warnings.warn(meta_warning(meta))


In [25]:
merged_df.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt,ETL_M,ADMI_CD,TOTAL_POPULATION,year,month,기준분기
0,202301,41210510,D05,633562,28,202301,41210510,13850.98,2023,1,1
1,202301,41210510,D11,171040,28,202301,41210510,13850.98,2023,1,1
2,202301,41210510,F02,12060758,375,202301,41210510,13850.98,2023,1,1
3,202301,41210510,Q01,1818710,9,202301,41210510,13850.98,2023,1,1
4,202301,41210510,Q15,2167723,87,202301,41210510,13850.98,2023,1,1


In [26]:
comm_df.head()

,행정동코드,기준분기,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분,상권변동지표구분명,year
0,41111560,1,99.9,46.4,HL,상권축소,2023
1,41111566,1,83.5,43.2,LL,다이나믹,2023
2,41111571,1,97.9,38.1,HL,상권축소,2023
3,41111572,1,92.9,58.5,HH,정체,2023
4,41111573,1,93.1,70.1,HH,정체,2023


In [27]:
# comm_df와 merged_df 병합
merged_result = dd.merge(merged_df, comm_df, left_on=['admi_cty_no', '기준분기', 'year'], right_on=['행정동코드', '기준분기', 'year'], how='left')

# 병합 결과에서 필요없는 열 제거 (예: '행정동코드')
merged_result = merged_result.drop(columns=['행정동코드'])

# 병합 결과 출력
merged_result.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt,ETL_M,ADMI_CD,TOTAL_POPULATION,year,month,기준분기,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분,상권변동지표구분명
0,202301,41210510,D05,633562,28,202301,41210510,13850.98,2023,1,1,188.5,358.0,HH,정체
1,202301,41210510,D11,171040,28,202301,41210510,13850.98,2023,1,1,188.5,358.0,HH,정체
2,202301,41210510,F02,12060758,375,202301,41210510,13850.98,2023,1,1,188.5,358.0,HH,정체
3,202301,41210510,Q01,1818710,9,202301,41210510,13850.98,2023,1,1,188.5,358.0,HH,정체
4,202301,41210510,Q15,2167723,87,202301,41210510,13850.98,2023,1,1,188.5,358.0,HH,정체


In [28]:
# 해당 행정동의 상권 지표를 결합
merged_df2 = dd.merge(merged_df, comm_df, how='inner', left_on=['admi_cty_no'], right_on=['행정동코드'])

In [29]:
merged_df2.columns

Index(['ta_ymd', 'admi_cty_no', 'card_tpbuz_cd', 'amt', 'cnt', 'ETL_M',
       'ADMI_CD', 'TOTAL_POPULATION', 'year_x', 'month', '기준분기_x', '행정동코드',
       '기준분기_y', '운영점포평균영업기간', '폐업점포평균영업기간', '상권변동지표구분', '상권변동지표구분명',
       'year_y'],
      dtype='object')

In [30]:
columns_to_drop = ['ta_ymd','ETL_M', 'ADMI_CD', 'year_x','기준분기_x', '행정동코드','기준분기_y',
                   '상권변동지표구분명', 'year_y','month']
merged_df2 = merged_df2.drop(columns=columns_to_drop)

In [103]:
merged_df2.head(10)

,admi_cty_no,card_tpbuz_cd,amt,cnt,TOTAL_POPULATION,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분
0,41210510,D05,633562,28,13850.98,188.5,358.0,HH
1,41210510,D05,633562,28,13850.98,188.5,358.0,HH
2,41210510,D05,633562,28,13850.98,188.5,358.0,HH
3,41210510,D11,171040,28,13850.98,188.5,358.0,HH
4,41210510,D11,171040,28,13850.98,188.5,358.0,HH
5,41210510,D11,171040,28,13850.98,188.5,358.0,HH
6,41210510,F02,12060758,375,13850.98,188.5,358.0,HH
7,41210510,F02,12060758,375,13850.98,188.5,358.0,HH
8,41210510,F02,12060758,375,13850.98,188.5,358.0,HH
9,41210510,Q01,1818710,9,13850.98,188.5,358.0,HH


In [32]:
# 505027 rows
len(merged_df2)

403080

In [105]:
merged_df2.nunique().sum()

164449

## One hot encoding, train-test split

In [54]:
merged_df2 = TabularDataset(merged_df2)

#### 인코딩시 사용되는 속성 저장

In [55]:
# merged_df2 = merged_df2.compute()

# # 범주형 변수의 모든 가능한 값 저장
# categorical_mapping = {
#     'card_tpbuz_cd': merged_df2['card_tpbuz_cd'].unique(),
#     '상권변동지표구분': merged_df2['상권변동지표구분'].unique()
# }
# with open('categorical_mapping.pkl', 'wb') as f:
#     pickle.dump(categorical_mapping, f)

# OneHotEncoder를 사용하여 범주형 변수를 인코딩
# categorical_columns = ["card_tpbuz_cd", "상권변동지표구분"]
# encoder = OneHotEncoder(sparse_output=False)
# encoded_categorical = encoder.fit_transform(merged_df2[categorical_columns])

# # 인코딩된 데이터를 DataFrame으로 변환하고 원래의 수치형 데이터와 결합
# encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_columns))
# train_data_encoded = pd.concat([merged_df2.drop(columns=categorical_columns), encoded_categorical_df], axis=1)

# with open('encoder.pkl', 'wb') as f:
#     pickle.dump(encoder, f)



In [58]:
# train_df, test_df = train_test_split(train_data_encoded, test_size=0.2, random_state=42)
train_df, test_df = train_test_split(merged_df2, test_size=0.2, random_state=42)

## 모델 학습1

In [60]:
# Autogluon TabularPredictor 사용
predictor = TabularPredictor(
    label='amt',  # 예측할 타겟 컬럼 설정
    eval_metric='mean_absolute_error'  # 평가 지표 설정 
)

# 모델 훈련
predictor.fit(
    train_data=train_df,
    presets='good_quality_faster_inference_only_refit',  # 가벼운 모델 설정
    time_limit=3600  # 학습 시간 2시간으로 제한 (1800초)
)

# 예측 수행
predictions = predictor.predict(test_df)

# 예측 결과 및 실제 값 비교
print("Predictions:\n", predictions)
print("Actual:\n", test_df['amt'])

No path specified. Models will be saved in: "AutogluonModels\ag-20240715_073451"
Preset alias specified: 'good_quality_faster_inference_only_refit' maps to 'good_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.8.18
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
Memory Avail:       13.54 GB / 31.69 GB (42.7%)
Disk Space Avail:   549.16 GB / 930.86 GB (59.0%)
Presets specified: ['good_quality_faster_inference_only_refit']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data

Predictions:
 388390    3.396393e+08
93442     1.850640e+07
396598    1.984676e+08
166263    1.558581e+08
88675     2.259014e+07
              ...     
155526    2.105532e+08
168829    1.460799e+09
153934    3.146199e+08
259348    2.911273e+07
310736    6.392836e+08
Name: amt, Length: 80616, dtype: float32
Actual:
 388390     365433405
93442       15489679
396598     193816450
166263     204928779
88675       23182820
             ...    
155526     183013621
168829    1543539840
153934     362683500
259348      29006955
310736     613019243
Name: amt, Length: 80616, dtype: int64


In [61]:
y_pred = predictor.predict(test_df.drop(columns=['amt']))
y_pred.head()

388390    339639264.0
93442      18506400.0
396598    198467616.0
166263    155858144.0
88675      22590136.0
Name: amt, dtype: float32

In [62]:
predictor.evaluate(test_df, silent=True)

{'mean_absolute_error': -24452663.372299213,
 'root_mean_squared_error': -169401599.57995763,
 'mean_squared_error': -2.86969019402483e+16,
 'r2': 0.9908262798078357,
 'pearsonr': 0.9961503003668908,
 'median_absolute_error': -7579201.5}

In [63]:
predictor.leaderboard(test_df)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE_BAG_L2_FULL,-2.407933e+07,NaN,mean_absolute_error,27.446944,NaN,376.674253,0.137518,2.427857,65.566918,2,True,24
1,WeightedEnsemble_L3_FULL,-2.445266e+07,NaN,mean_absolute_error,37.522038,NaN,413.887772,0.001998,NaN,1.452698,3,True,26
2,WeightedEnsemble_L2_FULL,-2.817289e+07,NaN,mean_absolute_error,11.538826,NaN,83.367764,0.011999,NaN,1.246012,2,True,21
3,LightGBM_BAG_L1_FULL,-2.902978e+07,NaN,mean_absolute_error,10.951635,NaN,30.234704,10.951635,NaN,30.234704,1,True,15
4,LightGBM_BAG_L2_FULL,-3.099782e+07,NaN,mean_absolute_error,37.382522,NaN,346.868156,10.073097,NaN,35.760821,2,True,23
5,RandomForestMSE_BAG_L1,-3.742274e+07,-3.981179e+07,mean_absolute_error,0.536744,8.230950,51.887049,0.536744,8.230950,51.887049,1,True,3
6,RandomForestMSE_BAG_L1_FULL,-3.742274e+07,NaN,mean_absolute_error,0.575191,8.230950,51.887049,0.575191,8.230950,51.887049,1,True,16
7,LightGBMXT_BAG_L2_FULL,-4.484916e+07,NaN,mean_absolute_error,40.248683,NaN,352.855522,12.939257,NaN,41.748187,2,True,22
8,LightGBMXT_BAG_L1_FULL,-5.201863e+07,NaN,mean_absolute_error,14.788396,NaN,36.050912,14.788396,NaN,36.050912,1,True,14
9,CatBoost_BAG_L2_FULL,-6.797594e+07,NaN,mean_absolute_error,27.330424,NaN,316.577955,0.020998,NaN,5.470620,2,True,25


In [48]:
train_df.to_csv('train_df.csv', index=False, encoding='utf-8-sig')
test_df.to_csv('test_df.csv', index=False, encoding='utf-8-sig')
merged_df2.to_csv('preprocessed.csv', index=False, encoding='utf-8-sig')

In [64]:
test_df.head()

,admi_cty_no,card_tpbuz_cd,amt,cnt,TOTAL_POPULATION,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분
388390,41590590,D01,365433405,906,7667999.44,37.7,33.8,LL
93442,41113560,D05,15489679,433,5216269.43,88.4,43.3,HL
396598,41590610,Y03,193816450,8579,8766419.48,27.1,28.7,LL
166263,41117580,Q04,204928779,7186,4852703.62,71.0,53.0,LL
88675,41113550,R01,23182820,75,7261073.44,82.7,63.0,LH


In [100]:
# 예측할 데이터
data = {
    "admi_cty_no": [41590590],
    "card_tpbuz_cd": ["D01"],
    "amt": [365433405],  # 예측에서는 사용되지 않음
    "cnt": [906],
    "TOTAL_POPULATION": [7667999.44	],
    "운영점포평균영업기간": [37.7],
    "폐업점포평균영업기간": [33.8],
    "상권변동지표구분": ["LL"]
}
# # 저장된 인코더 로드
# encoder = joblib.load('encoder.pkl')

df_predict = pd.DataFrame(data)
df_predict = TabularDataset(df_predict)

In [101]:
predictor2 = TabularPredictor.load(r"AutogluonModels\ag-20240715_073451")
y_pred = predictor2.predict(df_predict.drop(columns=['amt']))

In [102]:
y_pred

0    339639264.0
Name: amt, dtype: float32

In [75]:
df_predict

,admi_cty_no,card_tpbuz_cd,amt,cnt,TOTAL_POPULATION,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분
0,41210510,D06,633562,28,13850.98,188.5,358.0,HH


In [96]:
test_df[(test_df['card_tpbuz_cd'] == 'Y03') & (test_df['admi_cty_no'] == 41210510)]['amt'].mean()


1094938.0909090908

In [106]:
class AutogluonWrapper:
    def __init__(self, predictor, feature_names):
        self.ag_model = predictor
        self.feature_names = feature_names
    
    def predict_proba(self, X):
        if isinstance(X, pd.Series):
            X = X.values.reshape(1,-1)
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=self.feature_names)
        return self.ag_model.predict_proba(X)

In [2]:
%load_ext autoreload
%autoreload 2
!pip install -qU shap

In [3]:
import shap
shap.initjs()
import warnings
warnings.filterwarnings('ignore')
X_train = train_df.drop(columns=['amt'])
baseline = X_train.sample(100) 

ModuleNotFoundError: No module named 'shap'

In [119]:
ag_wrapper = AutogluonWrapper(predictor, X_train.columns)
explainer = shap.KernelExplainer(ag_wrapper.predict_proba, baseline)
print("Baseline prediction: ", np.mean(ag_wrapper.predict_proba(baseline)))  

NameError: name 'shap' is not defined

In [5]:
train_data

NameError: name 'train_data' is not defined